In [385]:
import pprint
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from json_shot_scraper import flatten_shot, flatten_goal, flatten_complete_pass, flatten_incomplete_pass, flatten_corner
from  player_scraper import flatten_player, flatten_sub
from dataframe_cleaner import (pass_to_shot, corner_to_shot, transpose_coordinates, coord_to_yards, 
                               shot_distance_angle, dummy_columns, drop_own_goals, goal_dummy, turn_into_dollar_value)
from model_prep import create_hypothetical_df
from transfer_market_scraper import flatten_player_info

In [4]:
pd.set_option('display.max_columns', 50)

In [63]:
from html_scraper import db
from mongo_to_db import game_to_cleaned_df, create_frame, create_master_df, create_master_player_min_df
import string

# players to match up data with

In [419]:
games = db.games.find()
players_minutes_df = create_master_player_min_df(games)

In [244]:
players_minutes_df['squad_number'].unique()

array([22., 12., 24.,  6., 29., 34.,  4., 15., 16., 30., 28.,  8., 23.,
       10., 11., 26.,  9., 33., nan, 13., 20., 21.,  5., 18., 31.,  7.,
       19.,  3., 14., 25.,  2., 27., 41., 40., 37., 32.,  1., 17., 39.,
       38., 87., 35., 36., 42., 43.])

In [420]:
players_minutes_df.head()

,game_id,name,player_id,position_id,squad_number,substitute,team_id,minutes_played,club_brev
0,448548,Alexander Domínguez,19132.0,1,22.0,False,20,90.0,VEL
1,448548,Lucas Hoyos,27987.0,1,12.0,True,20,0.0,VEL
2,448548,Gastón Díaz,30896.0,2,24.0,False,20,90.0,VEL
3,448548,Joaquín Laso,94225.0,2,6.0,False,20,90.0,VEL
4,448548,Luis Abram,99779.0,2,29.0,False,20,90.0,VEL


In [247]:
names = players_minutes_df['name'].unique()

In [291]:
afa_team_dict = {20: 'VEL', 13: 'NOB', 136: 'TIG', 19: 'SLO', 8: 'GIM', 2: 'ARG',
 137: 'UNI', 122: 'ALD', 869: 'PA', 6: 'COL', 124: 'BEL', 134: 'SMS',
 5: 'BOC', 135: 'TAL', 132: 'GOD', 7: 'EST', 12: 'LAN', 129: 'DEF',
 18: 'ROS', 4: 'BAN', 100: 'HUR', 17: 'RIV', 815: 'ATT', 16: 'RAC', 10: 'IND',
 490: 'SMT'}

In [292]:
len(afa_team_dict)

26

In [262]:
team_dict = [{'team_id': '20', 'team_intitials': 'VEL', 'short_name': 'Vélez'},
 {'team_id': '13', 'team_intitials': 'NOB', 'short_name': 'Newell`s'},
 {'team_id': '136', 'team_intitials': 'TIG', 'short_name': 'Tigre'},
 {'team_id': '19', 'team_intitials': 'SLO', 'short_name': 'San Lorenzo'},
 {'team_id': '8', 'team_intitials': 'GIM', 'short_name': 'Gimnasia'},
 {'team_id': '2', 'team_intitials': 'ARG', 'short_name': 'Argentinos'},
 {'team_id': '137', 'team_intitials': 'UNI', 'short_name': 'Unión'},
 {'team_id': '122', 'team_intitials': 'ALD', 'short_name': 'Aldosivi'},
 {'team_id': '869', 'team_intitials': 'PA', 'short_name': 'Patronato'},
 {'team_id': '6', 'team_intitials': 'COL', 'short_name': 'Colón'},
 {'team_id': '124', 'team_intitials': 'BEL', 'short_name': 'Belgrano'},
 {'team_id': '134', 'team_intitials': 'SMS', 'short_name': 'S. Martín SJ'},
 {'team_id': '5', 'team_intitials': 'BOC', 'short_name': 'Boca'},
 {'team_id': '135', 'team_intitials': 'TAL', 'short_name': 'Talleres'},
 {'team_id': '132', 'team_intitials': 'GOD', 'short_name': 'Godoy Cruz'},
 {'team_id': '7', 'team_intitials': 'EST', 'short_name': 'Estudiantes'},
 {'team_id': '12', 'team_intitials': 'LAN', 'short_name': 'Lanús'},
 {'team_id': '129', 'team_intitials': 'DEF', 'short_name': 'Defensa'},
 {'team_id': '18', 'team_intitials': 'ROS', 'short_name': 'R. Central'},
 {'team_id': '4', 'team_intitials': 'BAN', 'short_name': 'Banfield'},
 {'team_id': '100', 'team_intitials': 'HUR', 'short_name': 'Huracán'},
 {'team_id': '17', 'team_intitials': 'RIV', 'short_name': 'River'},
 {'team_id': '815', 'team_intitials': 'ATT', 'short_name': 'Atl. Tucumán'},
 {'team_id': '16', 'team_intitials': 'RAC', 'short_name': 'Racing'},
 {'team_id': '10', 'team_intitials': 'IND', 'short_name': 'Independiente'},
 {'team_id': '490', 'team_intitials': 'SMT', 'short_name': 'S. Martín T'}]

In [293]:
players_minutes_df['club_brev'] = players_minutes_df['team_id'].map(afa_team_dict)

In [295]:
players_minutes_df.head()

,game_id,name,player_id,position_id,squad_number,substitute,team_id,minutes_played,club_brev
0,448548,Alexander Domínguez,19132.0,1,22.0,False,20,90.0,VEL
1,448548,Lucas Hoyos,27987.0,1,12.0,True,20,0.0,VEL
2,448548,Gastón Díaz,30896.0,2,24.0,False,20,90.0,VEL
3,448548,Joaquín Laso,94225.0,2,6.0,False,20,90.0,VEL
4,448548,Luis Abram,99779.0,2,29.0,False,20,90.0,VEL


# Transfer Market Data

In [421]:
players = db.players.find()

In [422]:
master_list = []
for player in players:
    master_list.append(player)

In [423]:
tm_player = pd.DataFrame(master_list)


In [424]:
tm_player.head()

,_id,birthday,club,foot,height,player,squad_num,transfer_value(sterlings)
0,5c0588e63d54c00a19927577,"Jan 26, 1991 (27)",Squad Club Atlético Boca Juniors,right,"1,94 m",Esteban Andrada,31,£4.50m
1,5c0588e73d54c00a19927578,"Mar 17, 1987 (31)",Squad Club Atlético Boca Juniors,right,"1,92 m",Carlos Lampe,28,£495k
2,5c0588e73d54c00a19927579,"Sep 27, 1993 (25)",Squad Club Atlético Boca Juniors,right,"1,85 m",Lisandro Magallán,6,£4.05m
3,5c0588e73d54c00a1992757a,"May 12, 1985 (33)",Squad Club Atlético Boca Juniors,right,"1,84 m",Paolo Goltz,2,£1.62m
4,5c0588e73d54c00a1992757b,"Feb 22, 1991 (27)",Squad Club Atlético Boca Juniors,left,"1,73 m",Frank Fabra,18,£5.40m


### Turn squad numbers into floats...

In [311]:
tm_player['squad_num'].unique()

array(['31', '28', '6', '2', '18', '20', '29', '16', '15', '8', '30', '7',
       '22', '19', '17', '1', '25', '24', '26', '3', '14', '4', '40', '5',
       '39', '10', '11', '32', '9', '13', '-', '23', '21', '27', '41',
       '34', '37', '12', '36', '38', '33', '35', '87'], dtype=object)

In [312]:
tm_player.loc[tm_player.squad_num == '-', 'squad_num'] = None

In [313]:
tm_player['squad_num'].unique()

array(['31', '28', '6', '2', '18', '20', '29', '16', '15', '8', '30', '7',
       '22', '19', '17', '1', '25', '24', '26', '3', '14', '4', '40', '5',
       '39', '10', '11', '32', '9', '13', None, '23', '21', '27', '41',
       '34', '37', '12', '36', '38', '33', '35', '87'], dtype=object)

In [314]:
tm_player = tm_player.astype({'squad_num': float})

In [316]:
tm_player.head()

,_id,birthday,club,foot,height,player,squad_num,transfer_value(sterlings)
0,5c0588e63d54c00a19927577,"Jan 26, 1991 (27)",Squad Club Atlético Boca Juniors,right,"1,94 m",Esteban Andrada,31.0,£4.50m
1,5c0588e73d54c00a19927578,"Mar 17, 1987 (31)",Squad Club Atlético Boca Juniors,right,"1,92 m",Carlos Lampe,28.0,£495k
2,5c0588e73d54c00a19927579,"Sep 27, 1993 (25)",Squad Club Atlético Boca Juniors,right,"1,85 m",Lisandro Magallán,6.0,£4.05m
3,5c0588e73d54c00a1992757a,"May 12, 1985 (33)",Squad Club Atlético Boca Juniors,right,"1,84 m",Paolo Goltz,2.0,£1.62m
4,5c0588e73d54c00a1992757b,"Feb 22, 1991 (27)",Squad Club Atlético Boca Juniors,left,"1,73 m",Frank Fabra,18.0,£5.40m


### Add Column of Club_Abbrev

In [265]:
tm_player['club'].unique()

array(['Squad Club Atlético Boca Juniors',
       'Squad Club Atlético Independiente',
       'Squad Club Atlético River Plate', 'Squad Racing Club',
       'Squad Club Atlético San Lorenzo de Almagro',
       'Squad Club Atlético Lanús', 'Squad Club Atlético Vélez Sarsfield',
       'Squad Club Deportivo Godoy Cruz Antonio Tomba',
       'Squad Club Atlético Colón',
       'Squad Club Social y Deportivo Defensa y Justicia',
       'Squad Club Atlético Rosario Central',
       'Squad Club Atlético Talleres',
       'Squad Club Atlético Unión (Santa Fe)',
       'Squad Club Estudiantes de La Plata',
       'Squad Club Atlético Huracán',
       'Squad Asociación Atlética Argentinos Juniors',
       'Squad Club Atlético Banfield', 'Squad Club Atlético Belgrano',
       'Squad Club Atlético Tigre',
       "Squad Club Atlético Newell's Old Boys",
       'Squad Club Atlético Tucumán',
       'Squad Club de Gimnasia y Esgrima La Plata',
       'Squad Club Atlético Aldosivi',
       'Squad Clu

In [268]:
tm_club_brev = {'Squad Club Atlético Boca Juniors': 'BOC' , 'Squad Club Atlético Independiente': 'IND', 'Squad Club Atlético River Plate': 'RIV', 'Squad Racing Club': 'RAC',
       'Squad Club Atlético San Lorenzo de Almagro': 'SLO', 'Squad Club Atlético Lanús': 'LAN', 'Squad Club Atlético Vélez Sarsfield': 'VEL',
       'Squad Club Deportivo Godoy Cruz Antonio Tomba': 'GOD', 'Squad Club Atlético Colón': 'COL', 'Squad Club Social y Deportivo Defensa y Justicia': 'DEF',
       'Squad Club Atlético Rosario Central': 'ROS', 'Squad Club Atlético Talleres': 'TAL', 'Squad Club Atlético Unión (Santa Fe)': 'UNI',
       'Squad Club Estudiantes de La Plata': 'EST', 'Squad Club Atlético Huracán': 'HUR', 'Squad Asociación Atlética Argentinos Juniors': 'ARG',
       'Squad Club Atlético Banfield': 'BAN', 'Squad Club Atlético Belgrano': 'BEL', 'Squad Club Atlético Tigre': 'TIG', "Squad Club Atlético Newell's Old Boys": 'NOB',
       'Squad Club Atlético Tucumán': 'ATT', 'Squad Club de Gimnasia y Esgrima La Plata': 'GIM', 'Squad Club Atlético Aldosivi': 'ALD',
       'Squad Club Atlético San Martín (SJ)': 'SMS', 'Squad Club Atlético San Martín (Tucumán)': 'SMT', 'Squad Club Atlético Patronato de la Juventud Católica': 'PA'}

In [269]:
tm_player.head()

,_id,birthday,club,foot,height,player,squad_num,transfer_value(sterlings)
0,5c0588e63d54c00a19927577,"Jan 26, 1991 (27)",Squad Club Atlético Boca Juniors,right,"1,94 m",Esteban Andrada,31,£4.50m
1,5c0588e73d54c00a19927578,"Mar 17, 1987 (31)",Squad Club Atlético Boca Juniors,right,"1,92 m",Carlos Lampe,28,£495k
2,5c0588e73d54c00a19927579,"Sep 27, 1993 (25)",Squad Club Atlético Boca Juniors,right,"1,85 m",Lisandro Magallán,6,£4.05m
3,5c0588e73d54c00a1992757a,"May 12, 1985 (33)",Squad Club Atlético Boca Juniors,right,"1,84 m",Paolo Goltz,2,£1.62m
4,5c0588e73d54c00a1992757b,"Feb 22, 1991 (27)",Squad Club Atlético Boca Juniors,left,"1,73 m",Frank Fabra,18,£5.40m


In [317]:
tm_player['club_brev'] = tm_player['club'].map(tm_club_brev)

In [318]:
tm_player.head()

,_id,birthday,club,foot,height,player,squad_num,transfer_value(sterlings),club_brev
0,5c0588e63d54c00a19927577,"Jan 26, 1991 (27)",Squad Club Atlético Boca Juniors,right,"1,94 m",Esteban Andrada,31.0,£4.50m,BOC
1,5c0588e73d54c00a19927578,"Mar 17, 1987 (31)",Squad Club Atlético Boca Juniors,right,"1,92 m",Carlos Lampe,28.0,£495k,BOC
2,5c0588e73d54c00a19927579,"Sep 27, 1993 (25)",Squad Club Atlético Boca Juniors,right,"1,85 m",Lisandro Magallán,6.0,£4.05m,BOC
3,5c0588e73d54c00a1992757a,"May 12, 1985 (33)",Squad Club Atlético Boca Juniors,right,"1,84 m",Paolo Goltz,2.0,£1.62m,BOC
4,5c0588e73d54c00a1992757b,"Feb 22, 1991 (27)",Squad Club Atlético Boca Juniors,left,"1,73 m",Frank Fabra,18.0,£5.40m,BOC


## Test Merge

In [321]:
players_minutes_df.head()

,game_id,name,player_id,position_id,squad_number,substitute,team_id,minutes_played,club_brev
0,448548,Alexander Domínguez,19132.0,1,22.0,False,20,90.0,VEL
1,448548,Lucas Hoyos,27987.0,1,12.0,True,20,0.0,VEL
2,448548,Gastón Díaz,30896.0,2,24.0,False,20,90.0,VEL
3,448548,Joaquín Laso,94225.0,2,6.0,False,20,90.0,VEL
4,448548,Luis Abram,99779.0,2,29.0,False,20,90.0,VEL


In [323]:
players = players_minutes_df['player_id'].unique()
player_minutes = []
for player in players:
    total_minutes = players_minutes_df[players_minutes_df['player_id'] == player]['minutes_played'].sum()
    name = players_minutes_df[players_minutes_df['player_id'] == player]['name'].iloc[0]
    squad_num = players_minutes_df[players_minutes_df['player_id'] == player]['squad_number'].iloc[0]
    club_brev = players_minutes_df[players_minutes_df['player_id'] == player]['club_brev'].iloc[0]
    player_minutes.append([player, total_minutes, name, squad_num, club_brev])
    
summed_player_min = pd.DataFrame(player_minutes, columns=['player_id', 'total_minutes_played', 'player_name', 'squad_num', 'club_brev'])


In [325]:
summed_player_min.head()

,player_id,total_minutes_played,player_name,squad_num,club_brev
0,19132.0,1080.000000,Alexander Domínguez,22.0,VEL
1,27987.0,90.000000,Lucas Hoyos,12.0,VEL
2,30896.0,896.483333,Gastón Díaz,24.0,VEL
3,94225.0,990.000000,Joaquín Laso,6.0,VEL
4,99779.0,1080.000000,Luis Abram,29.0,VEL


In [327]:
player_merge_df = pd.merge(summed_player_min, tm_player, on=['club_brev', 'squad_num'])

In [329]:
player_merge_df.drop(columns=['_id', 'club_brev'], inplace=True)

In [335]:
columns = ['player_id', 'player_name', 'foot', 'birthday', 'club', 'transfer_value(sterlings)', 'total_minutes_played']

In [337]:
player_merge = player_merge_df[columns].copy()

In [338]:
player_merge.head()

,player_id,player_name,foot,birthday,club,transfer_value(sterlings),total_minutes_played
0,19132.0,Alexander Domínguez,right,"Jun 5, 1987 (31)",Squad Club Atlético Vélez Sarsfield,£1.58m,1080.000000
1,27987.0,Lucas Hoyos,right,"Apr 29, 1989 (29)",Squad Club Atlético Vélez Sarsfield,£135k,90.000000
2,30896.0,Gastón Díaz,right,"Mar 13, 1988 (30)",Squad Club Atlético Vélez Sarsfield,£450k,896.483333
3,94225.0,Joaquín Laso,right,"Jul 4, 1990 (28)",Squad Club Atlético Vélez Sarsfield,£270k,990.000000
4,99779.0,Luis Abram,left,"Feb 27, 1996 (22)",Squad Club Atlético Vélez Sarsfield,£405k,1080.000000


### pulling out age 

In [410]:
player_merge['age'] = player_merge.birthday.str[-3:-1].astype(int)

In [411]:
player_merge.tail(20)

,player_id,player_name,foot,birthday,club,transfer_value(sterlings),total_minutes_played,age,transfer_value(USD)
608,63197.0,Fernando Brandán,right,"Mar 27, 1990 (28)",Squad Club Atlético San Martín (SJ),£495k,0.000000,28,0.63
609,106312.0,Lucas Acosta,right,"Mar 12, 1995 (23)",Squad Club Atlético Belgrano,£450k,0.000000,23,0.57
610,120431.0,Alexis Soto,left,"Oct 20, 1993 (25)",Squad Racing Club,£2.52m,148.850000,25,3.2
611,204560.0,Agustín Bolívar,right,"Jan 9, 1996 (22)",Squad Club de Gimnasia y Esgrima La Plata,£68k,0.000000,22,0.09
612,69819.0,Dardo Miloc,right,"Oct 16, 1990 (28)",Squad Club Atlético Aldosivi,£315k,225.000000,28,0.4
613,45639.0,David Drocco,right,"Jan 20, 1989 (29)",Squad Club Atlético Huracán,£225k,5.833333,29,0.29
614,221332.0,Agustín Casco,right,"May 25, 1993 (25)",Squad Club Atlético Huracán,£495k,10.250000,25,0.63
615,147292.0,Gastón Gómez,right,"Mar 4, 1996 (22)",Squad Racing Club,£450k,0.000000,22,0.57
616,204641.0,Fabricio Brener,right,"May 26, 1998 (20)",Squad Club Atlético Belgrano,£135k,13.466667,20,0.17
617,208582.0,Kevin Gutiérrez,right,"Jun 3, 1997 (21)",Squad Club de Gimnasia y Esgrima La Plata,-,0.000000,21,-


## turning transfer_value into dollars

In [416]:
player_merge['transfer_value(USD)'] = player_merge['transfer_value(sterlings)'].apply(lambda x: turn_into_dollar_value(x, 1.27))

In [402]:
player_merge.drop(columns = ['player_value', 'player_value(USD)'], inplace=True)

In [417]:
player_merge.head()

,player_id,player_name,foot,birthday,club,transfer_value(sterlings),total_minutes_played,age,transfer_value(USD)
0,19132.0,Alexander Domínguez,right,"Jun 5, 1987 (31)",Squad Club Atlético Vélez Sarsfield,£1.58m,1080.000000,31,2.01
1,27987.0,Lucas Hoyos,right,"Apr 29, 1989 (29)",Squad Club Atlético Vélez Sarsfield,£135k,90.000000,29,0.17
2,30896.0,Gastón Díaz,right,"Mar 13, 1988 (30)",Squad Club Atlético Vélez Sarsfield,£450k,896.483333,30,0.57
3,94225.0,Joaquín Laso,right,"Jul 4, 1990 (28)",Squad Club Atlético Vélez Sarsfield,£270k,990.000000,28,0.34
4,99779.0,Luis Abram,left,"Feb 27, 1996 (22)",Squad Club Atlético Vélez Sarsfield,£405k,1080.000000,22,0.51


In [418]:
player_merge[(player_merge['age'] < 25) & (player_merge['transfer_value(USD)'] > 5)]

,player_id,player_name,foot,birthday,club,transfer_value(sterlings),total_minutes_played,age,transfer_value(USD)
15,177285.0,Matías Vargas,right,"May 8, 1997 (21)",Squad Club Atlético Vélez Sarsfield,£8.10m,1024.733333,21,10.29
120,93853.0,Franco Soldano,right,"Sep 14, 1994 (24)",Squad Club Atlético Unión (Santa Fe),£4.50m,1060.350000,24,5.71
222,119150.0,Nahitan Nández,right,"Dec 28, 1995 (22)",Squad Club Atlético Boca Juniors,£8.55m,441.616667,22,10.86
226,126655.0,Emanuel Reynoso,left,"Nov 16, 1995 (23)",Squad Club Atlético Boca Juniors,£5.40m,85.066667,23,6.86
227,111421.0,Cristian Pavón,right,"Jan 21, 1996 (22)",Squad Club Atlético Boca Juniors,£18.00m,553.933333,22,22.86
378,153417.0,Lucas Martínez Quarta,right,"May 10, 1996 (22)",Squad Club Atlético River Plate,£5.85m,360.000000,22,7.43
428,100773.0,Fabricio Bustos,right,"Apr 28, 1996 (22)",Squad Club Atlético Independiente,£9.00m,868.733333,22,11.43
429,105543.0,Nicolás Figal,right,"Apr 3, 1994 (24)",Squad Club Atlético Independiente,£4.05m,626.850000,24,5.14
467,193185.0,Alan Franco,right,"Oct 11, 1996 (22)",Squad Club Atlético Independiente,£5.40m,404.216667,22,6.86
470,79877.0,Martín Benítez,right,"Jun 17, 1994 (24)",Squad Club Atlético Independiente,£5.40m,393.233333,24,6.86


## Testing cleaning functions

In [425]:
games = db.games.find()
players_minutes_df = create_master_player_min_df(games)

In [426]:
players_minutes_df.head()

,game_id,name,player_id,position_id,squad_number,substitute,team_id,minutes_played,club_brev
0,448548,Alexander Domínguez,19132.0,1,22.0,False,20,90.0,VEL
1,448548,Lucas Hoyos,27987.0,1,12.0,True,20,0.0,VEL
2,448548,Gastón Díaz,30896.0,2,24.0,False,20,90.0,VEL
3,448548,Joaquín Laso,94225.0,2,6.0,False,20,90.0,VEL
4,448548,Luis Abram,99779.0,2,29.0,False,20,90.0,VEL


In [445]:
from model_prep import player_minutes_total

In [446]:
summed_player_min = player_minutes_total(players_minutes_df)

In [447]:
summed_player_min.head()

,player_id,total_minutes_played,player_name,squad_num,club_brev
0,19132.0,1080.000000,Alexander Domínguez,22.0,VEL
1,27987.0,90.000000,Lucas Hoyos,12.0,VEL
2,30896.0,896.483333,Gastón Díaz,24.0,VEL
3,94225.0,990.000000,Joaquín Laso,6.0,VEL
4,99779.0,1080.000000,Luis Abram,29.0,VEL


In [427]:
from dataframe_cleaner import transfer_markt_cleaner

In [437]:
players = db.players.find()

In [438]:
tm_df = transfer_markt_cleaner(players)

In [439]:
tm_df.head()

,_id,birthday,club,foot,height,player,squad_num,transfer_value(sterlings),club_brev,age,transfer_value(USD)
0,5c0588e63d54c00a19927577,"Jan 26, 1991 (27)",Squad Club Atlético Boca Juniors,right,"1,94 m",Esteban Andrada,31.0,£4.50m,BOC,27,5.71
1,5c0588e73d54c00a19927578,"Mar 17, 1987 (31)",Squad Club Atlético Boca Juniors,right,"1,92 m",Carlos Lampe,28.0,£495k,BOC,31,0.63
2,5c0588e73d54c00a19927579,"Sep 27, 1993 (25)",Squad Club Atlético Boca Juniors,right,"1,85 m",Lisandro Magallán,6.0,£4.05m,BOC,25,5.14
3,5c0588e73d54c00a1992757a,"May 12, 1985 (33)",Squad Club Atlético Boca Juniors,right,"1,84 m",Paolo Goltz,2.0,£1.62m,BOC,33,2.06
4,5c0588e73d54c00a1992757b,"Feb 22, 1991 (27)",Squad Club Atlético Boca Juniors,left,"1,73 m",Frank Fabra,18.0,£5.40m,BOC,27,6.86


In [448]:
player_merge_df = pd.merge(summed_player_min, tm_df, on=['club_brev', 'squad_num'])

In [450]:
player_merge_df.head()

,player_id,total_minutes_played,player_name,squad_num,club_brev,_id,birthday,club,foot,height,player,transfer_value(sterlings),age,transfer_value(USD)
0,19132.0,1080.000000,Alexander Domínguez,22.0,VEL,5c05891a3d54c00a19927619,"Jun 5, 1987 (31)",Squad Club Atlético Vélez Sarsfield,right,"1,95 m",Alexander Domínguez,£1.58m,31,2.01
1,27987.0,90.000000,Lucas Hoyos,12.0,VEL,5c05891a3d54c00a19927626,"Apr 29, 1989 (29)",Squad Club Atlético Vélez Sarsfield,right,"1,83 m",Lucas Hoyos,£135k,29,0.17
2,30896.0,896.483333,Gastón Díaz,24.0,VEL,5c05891a3d54c00a1992761e,"Mar 13, 1988 (30)",Squad Club Atlético Vélez Sarsfield,right,"1,75 m",Gastón Díaz,£450k,30,0.57
3,94225.0,990.000000,Joaquín Laso,6.0,VEL,5c05891a3d54c00a1992761b,"Jul 4, 1990 (28)",Squad Club Atlético Vélez Sarsfield,right,"1,85 m",Joaquín Laso,£270k,28,0.34
4,99779.0,1080.000000,Luis Abram,29.0,VEL,5c05891a3d54c00a19927627,"Feb 27, 1996 (22)",Squad Club Atlético Vélez Sarsfield,left,"1,80 m",Luis Abram,£405k,22,0.51


In [451]:
player_merge_df.drop(columns=['_id', 'club_brev'], inplace=True)

In [452]:
player_merge_df.head()

,player_id,total_minutes_played,player_name,squad_num,birthday,club,foot,height,player,transfer_value(sterlings),age,transfer_value(USD)
0,19132.0,1080.000000,Alexander Domínguez,22.0,"Jun 5, 1987 (31)",Squad Club Atlético Vélez Sarsfield,right,"1,95 m",Alexander Domínguez,£1.58m,31,2.01
1,27987.0,90.000000,Lucas Hoyos,12.0,"Apr 29, 1989 (29)",Squad Club Atlético Vélez Sarsfield,right,"1,83 m",Lucas Hoyos,£135k,29,0.17
2,30896.0,896.483333,Gastón Díaz,24.0,"Mar 13, 1988 (30)",Squad Club Atlético Vélez Sarsfield,right,"1,75 m",Gastón Díaz,£450k,30,0.57
3,94225.0,990.000000,Joaquín Laso,6.0,"Jul 4, 1990 (28)",Squad Club Atlético Vélez Sarsfield,right,"1,85 m",Joaquín Laso,£270k,28,0.34
4,99779.0,1080.000000,Luis Abram,29.0,"Feb 27, 1996 (22)",Squad Club Atlético Vélez Sarsfield,left,"1,80 m",Luis Abram,£405k,22,0.51


In [454]:
player_merge_df[player_merge_df['transfer_value(USD)'] > 10]

,player_id,total_minutes_played,player_name,squad_num,birthday,club,foot,height,player,transfer_value(sterlings),age,transfer_value(USD)
15,177285.0,1024.733333,Matías Vargas,26.0,"May 8, 1997 (21)",Squad Club Atlético Vélez Sarsfield,right,"1,68 m",Matías Vargas,£8.10m,21,10.29
222,119150.0,441.616667,Nahitan Nández,15.0,"Dec 28, 1995 (22)",Squad Club Atlético Boca Juniors,right,"1,72 m",Nahitan Nández,£8.55m,22,10.86
223,97067.0,450.000000,Wilmar Barrios,16.0,"Oct 16, 1993 (25)",Squad Club Atlético Boca Juniors,right,"1,79 m",Wilmar Barrios,£9.00m,25,11.43
227,111421.0,553.933333,Cristian Pavón,7.0,"Jan 21, 1996 (22)",Squad Club Atlético Boca Juniors,right,"1,74 m",Cristian Pavón,£18.00m,22,22.86
382,77919.0,321.700000,Gonzalo Martínez,10.0,"Jun 13, 1993 (25)",Squad Club Atlético River Plate,left,"1,70 m",Gonzalo Martínez,£13.50m,25,17.14
428,100773.0,868.733333,Fabricio Bustos,16.0,"Apr 28, 1996 (22)",Squad Club Atlético Independiente,right,"1,67 m",Fabricio Bustos,£9.00m,22,11.43
438,85466.0,759.100000,Maximiliano Meza,8.0,"Jan 15, 1992 (26)",Squad Club Atlético Independiente,right,"1,80 m",Maximiliano Meza,£12.60m,26,16.00
550,40071.0,305.883333,Darío Benedetto,9.0,"May 17, 1990 (28)",Squad Club Atlético Boca Juniors,right,"1,77 m",Darío Benedetto,£9.00m,28,11.43


In [455]:
player_merge_df.columns

Index(['player_id', 'total_minutes_played', 'player_name', 'squad_num',
       'birthday', 'club', 'foot', 'height', 'player',
       'transfer_value(sterlings)', 'age', 'transfer_value(USD)'],
      dtype='object')

In [458]:
columns = ['player_id', 'player_name', 'club', 'birthday', 'age', 'foot', 'squad_num',
           'height', 'total_minutes_played', 'transfer_value(sterlings)', 'transfer_value(USD)']

# all together

In [460]:
from combined_player import player_minutes_value

In [464]:
games = db.games.find()
players = db.players.find()

In [465]:
final_df = player_minutes_value(games, players)

In [466]:
final_df

,player_id,player_name,club,birthday,age,foot,squad_num,height,total_minutes_played,transfer_value(sterlings),transfer_value(USD)
0,19132.0,Alexander Domínguez,Squad Club Atlético Vélez Sarsfield,"Jun 5, 1987 (31)",31,right,22.0,"1,95 m",1080.000000,£1.58m,2.01
1,27987.0,Lucas Hoyos,Squad Club Atlético Vélez Sarsfield,"Apr 29, 1989 (29)",29,right,12.0,"1,83 m",90.000000,£135k,0.17
2,30896.0,Gastón Díaz,Squad Club Atlético Vélez Sarsfield,"Mar 13, 1988 (30)",30,right,24.0,"1,75 m",896.483333,£450k,0.57
3,94225.0,Joaquín Laso,Squad Club Atlético Vélez Sarsfield,"Jul 4, 1990 (28)",28,right,6.0,"1,85 m",990.000000,£270k,0.34
4,99779.0,Luis Abram,Squad Club Atlético Vélez Sarsfield,"Feb 27, 1996 (22)",22,left,29.0,"1,80 m",1080.000000,£405k,0.51
5,208534.0,Francisco Ortega,Squad Club Atlético Vélez Sarsfield,"Feb 19, 1999 (19)",19,left,34.0,"1,77 m",360.000000,£360k,0.46
6,208462.0,Hernán De La Fuente,Squad Club Atlético Vélez Sarsfield,"Jan 7, 1997 (21)",21,right,4.0,"1,78 m",270.000000,£540k,0.69
7,193952.0,Nahuel Arena,Squad Club Atlético Vélez Sarsfield,"Jun 2, 1998 (20)",20,right,15.0,"1,83 m",-1.450000,-,NaN
8,185745.0,Lucas Robertone,Squad Club Atlético Vélez Sarsfield,"Mar 18, 1997 (21)",21,right,16.0,"1,70 m",881.500000,£2.70m,3.43
9,60730.0,Gastón Giménez,Squad Club Atlético Vélez Sarsfield,"Jul 27, 1991 (27)",27,left,30.0,"1,88 m",1126.450000,£1.80m,2.29
